In [ ]:
import numpy as np
from pathlib import Path
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gdf

from hydromt.data_catalog import DataCatalog
from hydromt_sfincs import SfincsModel
from hydromt_sfincs.utils import downscale_floodmap

from flood_adapt.api import startup

In [ ]:
# Path and name of FloodAdapt database
db_path = Path("/mnt/c/Repositories/DT-flood/FloodAdapt_database")
db_name = "Humber"
# Read database
db = startup.read_database(database_path=db_path, site_name=db_name)

# Name of the scenario to visualize
scenario_name = "Empty_Event_current_no_measures"
# Read scenario
scenario = db.get_scenario(name=scenario_name)
# Get output path
out_path = scenario.results_path

# Set some useful paths
floodmap_fn = out_path/"Flooding"/f"FloodMap_{scenario_name}.tif"
sf_path = out_path/"Flooding"/"simulations"/"overland"

# Init data catalog
data_catalogues = [Path("/mnt/c/Repositories/DT-flood/Data/gtsm_catalogue.yml"), Path("/home/wotromp/DestinE_workflows/deltares_data_wsl.yml")]
dc = DataCatalog(data_libs=data_catalogues)

region = gdf.read_file(db_path/db_name/"HumberDelta_large.geojson")

In [ ]:
sf = SfincsModel(
    root=sf_path,
    mode='r',
    data_libs=data_catalogues,
)
sf.read_results()
dep_da = xr.open_dataset(db.static_path/"dem"/"Humber_subgrid.tif")

In [ ]:
proj = ccrs.Projection(sf.crs)

In [ ]:
dep_da['band_data'].plot()

In [ ]:
sf.plot_basemap()

In [ ]:
gswo = dc.get_rasterdataset("gswo", geom=sf.region, buffer=100)

In [ ]:
gswo_mask = gswo.raster.reproject_like(sf.grid, method="max") <= 5

In [ ]:
hmin= 0.01
hmax = downscale_floodmap(
    zsmax=sf.results['zsmax'].where(gswo_mask).where(sf.results['zsmax']>hmin),
    dep=dep_da['band_data'],
)

In [ ]:
fig = plt.figure(figsize=(14,7))
ax = fig.add_subplot(projection=proj)
ax.add_image(cimgt.QuadtreeTiles(),10, interpolation='bilinear')
hmax.plot(ax=ax,vmin=0, vmax=3, cmap='Blues')
ax.set_extent(region.total_bounds[[0,2,1,3]])
ax.xaxis.set_visible(True)
ax.yaxis.set_visible(True)
ax.set_title(f"Floodmap for event {scenario_name}")

In [ ]:
sf.forcing.keys()

In [ ]:
sf.forcing['press_2d'].isel(time=67).plot.contour()

In [ ]:
sf.forcing['press_2d'].min().values

In [ ]:
from matplotlib import animation

da = sf.forcing['press_2d']

step = 1

vmin, vmax = da.quantile([0., 0.98]).compute()
c_dem = plt.cm.viridis(np.linspace(0.25, 1, 512))
cmap = colors.LinearSegmentedColormap.from_list("dem", c_dem)
norm = colors.Normalize(vmin=vmin, vmax=vmax)
cmap_kwargs = dict(cmap=cmap, norm=norm) 

# def update_plot(i, da_h, cax_h):
#     da_hi = da_h.isel(time=i)
#     t = da_hi.time.dt.strftime("%d-%B-%Y %H:%M:%S").item()
#     ax.set_title(f"Pressure Forcing {t}, max values: {da_hi.max().values}")
#     cax_h.set_array(da_hi.values.ravel())
#     cax_h.set_clim(vmin=da_hi.min().values, vmax=da_hi.max().values)

def update_plot(i, da_h):
    global cax_h, cl
    da_hi = da_h.isel(time=i)
    for c in cax_h.collections:
        c.remove()
    for l in cl:
        l.remove()
    cax_h = da_hi.plot.contour(
        x='x', y='y', ax=ax,
        vmin=da.min().values, vmax=da.max().values, levels=100
    )
    # cax_h.set_clim(vmin=da_h.min().values, vmax=da_h.max().values)
    cl = ax.clabel(cax_h)
    return cax_h

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(projection=proj)
ax.add_image(cimgt.OSM(),10, interpolation='bilinear')

cax_h = da.isel(time=0).plot.contourf(
    x='x', y='y',
    ax=ax,
    vmin=da.min().values, vmax=da.max().values, levels=100,
    # add_colorbar=True,
    # cbar_kwargs=cbar_kwargs
)
# cbar = fig.colorbar(cax_h)

for c in cax_h.collections:
    c.remove()

cax_h = da.isel(time=0).plot.contour(
    x='x', y='y',
    ax=ax,
    vmin=da.min().values, vmax=da.max().values, levels=100,
    # add_colorbar=True,
    # cbar_kwargs=cbar_kwargs
)
cl = ax.clabel(cax_h)


ani = animation.FuncAnimation(
    fig,
    update_plot,
    frames=np.arange(0,da.time.size,step),
    interval=250,
    fargs=[da]
)

plt.close()

# ani.save('test_forcing_animation.gif', fps=4, dpi=200)

#show in notebook
from IPython.display import HTML

HTML(ani.to_jshtml())

In [ ]:
c_dem.shape